In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
import argparse
import sys


####providing sample picture set####
from tensorflow.examples.tutorials.mnist import input_data
import tensorflow as tf
FLAGS = None

In [2]:
if __name__ == '__main__':
  parser = argparse.ArgumentParser()
  parser.add_argument('--data_dir', type=str, default='/tmp/tensorflow/mnist/input_data',
                      help='Directory for storing input data')
  FLAGS, unparsed = parser.parse_known_args()

In [3]:
mnist = input_data.read_data_sets(FLAGS.data_dir, one_hot=True)

Extracting /tmp/tensorflow/mnist/input_data/train-images-idx3-ubyte.gz
Extracting /tmp/tensorflow/mnist/input_data/train-labels-idx1-ubyte.gz
Extracting /tmp/tensorflow/mnist/input_data/t10k-images-idx3-ubyte.gz
Extracting /tmp/tensorflow/mnist/input_data/t10k-labels-idx1-ubyte.gz


In [4]:
###CNN Addon - weights created with a little bit of noise (symmetry braking?) 
#and bias > 0 (i.e. 0.1) to avoid dead neurons (?)
def weight_variable(shape):
  initial = tf.truncated_normal(shape, stddev=0.1)
  return tf.Variable(initial)

def bias_variable(shape):
  initial = tf.constant(0.1, shape=shape)
  return tf.Variable(initial)

#convolution functions, strides (offset?) is kept at one, padding is nothing (input equals output)

def conv2d(x, W):
  return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
  return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME')

#x = tf.placeholder(tf.float32, [None, 784])
#dimension NONE so that it can be any size according to the batch size#
x = tf.placeholder(tf.float32, shape=[None, 784])
#writing x with a shape argument helps to catch bugs stemming from inconcsistent tensor shapes#
W = tf.Variable(tf.zeros([784, 10]))
b = tf.Variable(tf.zeros([10]))
y = tf.matmul(x, W) + b

###first layer of convolution + max pooling ###
#dimensions: patch size, patch size, input channels, number of output channels. 
#bias vector with component for each output channel.
W_conv1 = weight_variable([5, 5, 1, 32])
b_conv1 = bias_variable([32])

#x needs to be reshaped to a 4D tensor to apply the convolution layer
#dimensions: ?, img h, img w, input channels
x_image = tf.reshape(x, [-1,28,28,1])


#We then convolve x_image with the weight tensor, 
#add the bias, apply the ReLU function, and finally max pool. 
#The max_pool_2x2 method will reduce the image size to 14x14.

h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

###SECOND CONVOLUTIONAL LAYER###
#In order to build a deep network, we stack several layers of this type. 
#The second layer will have 64 features for each 5x5 patch.

W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])
h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

###DENSELY CONNECTED LAYER###
#Now that the image size has been reduced to 7x7, we add a fully-connected layer 
#with 1024 neurons to allow processing on the entire image. 
#We reshape the tensor from the pooling layer into a batch of vectors, 
#multiply by a weight matrix, add a bias, and apply a ReLU.

W_fc1 = weight_variable([7 * 7 * 64, 1024])
b_fc1 = bias_variable([1024])

h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

####DROPOUT####
#can be turned off during testing
keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)


####FINAL READOUT Layer####
W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])
y_conv = tf.matmul(h_fc1_drop, W_fc2) + b_fc2

# Define loss and optimizer
#output class (to be one hot encoded to tell us which digit label we have found)
y_ = tf.placeholder(tf.float32, shape=[None, 10])

#loss function telling us how bad the initial guess war (comparing y true vs y_ (predict))#
#cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(y, y_))
#reducing the error by applying gradient descent (ableitungen)#
#train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(y_conv, y_))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [ ]:
#instead of defining the graph first and then running the session, we use an interactive sessions#
#it ought to give us more flexibility in designing the graph#
sess = tf.InteractiveSession()
#builds the tensor for all variables created:
tf.global_variables_initializer().run()
###computation initialized here will run outside python (c++?)#

sess.run(tf.global_variables_initializer())
for i in range(500):
  batch = mnist.train.next_batch(50)
  if i%100 == 0:
    train_accuracy = accuracy.eval(feed_dict={
        x:batch[0], y_: batch[1], keep_prob: 1.0})
    print("step %d, training accuracy %g"%(i, train_accuracy))
  train_step.run(feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})

print("test accuracy %g"%accuracy.eval(feed_dict={
    x: mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0}))



step 0, training accuracy 0.12
step 100, training accuracy 0.8
step 200, training accuracy 0.92
step 300, training accuracy 0.92
step 400, training accuracy 0.98


In [12]:
#tf.app.run(main=main, argv=[sys.argv[0]] + unparsed)

SystemExit: 

In [13]:
 # Test trained model
    #argmax gives index of highest entry within the defined axis of a tensor#
    #tf equal checks if prediction (y_) is the same as the real label (y)#
#correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
#accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
#print(sess.run(accuracy, feed_dict={x: mnist.test.images,y_: mnist.test.labels}))

0.9154


In [21]:
#mnist.train.labels[1]

array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.])

In [29]:
#mnist.train.images[1]
image_size = mnist.test.images[0].shape[0]
print ('image_size => {0}'.format(image_size))

# in this case all images are square
image_width = image_height = np.ceil(np.sqrt(image_size)).astype(np.uint8)

print ('image_width => {0}\nimage_height => {1}'.format(image_width,image_height))

image_size => 784
image_width => 28
image_height => 28


In [49]:
import matplotlib.cm as cm
def display(img):
    
    # (784) => (28,28)
    one_image = img.reshape(image_width,image_height)
    

    plt.axis('off')
    plt.imshow(one_image, cmap=cm.binary,)

#display(mnist.test.images[23])
